<a href="https://colab.research.google.com/github/alcarvalhox/Projeto-Eddy-Current/blob/master/EC_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
import datetime
import math
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.io
from scipy import signal
from scipy.signal import medfilt
import seaborn as sns
import pywt
import csv

#################################################
from google.colab import drive
drive.mount("/content/drive/")

#################################################
try:
  path = '/content/drive/My Drive/Projeto Eddy Current/ProjetoFinal'
  os.chdir(path)
except:
  print("Erro ao carregar diretorio no drive. Usado para drive compartilhado")

#################################################
print("Sistema preparado para rodar!")
print(datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S"))

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
Sistema preparado para rodar!
20-08-2020 18:18:50


# **Bloco de Classes**

In [48]:


#######################################
#### Leitura e Abertura de Arquivo ####
#######################################

class carrega_arquivo:
  def __init__(self):
    pass 

  def le_arquivo (self ,Path,File):
    #### Lê Arquivo .txt 
    with open(Path +'/'+ File, 'r') as f:
      # Usando a expressão regular para o separador indicando que eles são os espaços em branco
      df = pd.read_table(f, sep='\s+')
      df=df.replace(np.nan,0)
            

        # lista os cabeçalhos    
      cabecalho=df.columns

        # atribui a coluna do gráfico no indíce da coluna
      c = cabecalho[2]
        # define o comprimento da coluna
      a = len(df[cabecalho[2]])
        
        # cria vetor com quantidade de pontos
      x = np.arange(a)
        
        # define quantidade de linhas (pontos) e colunas
      linha = a
      coluna = 1
      
      return df[c]


##############################################################

#######################################
####        Filtra Sinais          ####
#######################################



class filtro:
  def __init__(self):
    pass

############### Spikes ################

  def tira_spikes(self,teste):

    resultado = []
    grad = (np.gradient(teste))
    valores = [int(val) for val in grad]

    for x in range (1,len(teste)):
        resultado.append(teste[x])

    for x in range (1,len(teste)):
        if abs(valores[x]) > (200):
            for x in range (x,(x+500)):
                resultado[x]=0 # Sinal Original Filtrado 
                
    resultado=signal.medfilt(resultado,kernel_size=11) 
    
    return resultado

############### Filtro  Media ################
  def filtro_media(self,sinalf,picos,janela2):

    (n,qsinais) = sinalf.shape  
    sinal=sinalf 
    contsinais =0
    janela4 = []
    picos4 = []

    for x in range(0,qsinais):
        if (np.mean(sinal[:,x])> mediainf) and (np.mean(sinal[:,x])< mediasup):
            janela4.append(janela2[x])
            picos4.append(picos[x])
            contsinais=contsinais+1
            
    #%%%% Cria matriz sinalfinal        
    sinalfinal = np.zeros(((amostras+1),(contsinais)))    

    #%%%% Coloca sinais filtrados por média na matriz sinalfinal
    qsinais2=0
    for x in range(0,(contsinais)):
        if (np.mean(sinal[:,x])> mediainf) and (np.mean(sinal[:,x])< mediasup):
            
            sinalfinal[:,qsinais2]=sinal[:,x]
            qsinais2=qsinais2+1

    return sinalfinal,janela4,picos4

  ############### Filtro  FFT ################
  def filtro_fft(self,sinalfinal,picos4,janela4):  

    (n,qsinais3) = sinalfinal.shape
    contasinais2 = 0
    janela5 = []
    picos5 = []

    for x in range(0,qsinais3):
        a=(max(np.fft.fft(sinalfinal[:,x])))
        #if a.imag != 0:
        if a.imag < 2000:    
            janela5.append(janela4[x])
            picos5.append(picos4[x])
            contasinais2=contasinais2+1
            
    # Cria matriz sinalfinal2
    sinalfinal2 = np.zeros(((amostras+1),(contasinais2)))

    # Coloca sinais filtrados por FFT na matriz sinalfinal2
    qsinais4 = 0

    for x in range(0,qsinais3):
        a=(max(np.fft.fft(sinalfinal[:,x])))
        #if a.imag != 0:
        if a.imag < 2000:    
            sinalfinal2[:,qsinais4] = sinalfinal[:,x]
            qsinais4=qsinais4+1    

    return sinalfinal2,janela5,picos5
  

################ Filtro Zerocross #################
  def filtro_zerocross(self,sinalfinal2,picos5,janela5):

    (n,qsinais5) = sinalfinal2.shape
    contasinais3 = 0
    janela6 = []
    picos6 = []
    zero_x = 0

    for x in range(0,qsinais5):
        zero_x = 0
        for idx in range(1,(amostras-1)):
            if sinalfinal2[idx,x] < 0 and sinalfinal2[(idx+1),x] > 0:
                zero_x +=1
            if sinalfinal2[idx,x] > 0 and sinalfinal2[(idx+1),x] < 0:
                zero_x +=1
                
        if zero_x > zerocross:
            janela6.append(janela5[x])
            picos6.append(picos5[x])
            contasinais3=contasinais3+1
            
    # Cria matriz sinalfinal2
    sinalfinal3 = np.zeros(((amostras+1),(contasinais3)))

    # Coloca sinais filtrados por ZeroCross na matriz sinalfinal2
    qsinais6 = 0

    for x in range(0,qsinais5):
        zero_x = 0
        for idx in range(1,(amostras-1)):
            if sinalfinal2[idx,x] < 0 and sinalfinal2[(idx+1),x] > 0:
                zero_x +=1
            if sinalfinal2[idx,x] > 0 and sinalfinal2[(idx+1),x] < 0:
                zero_x +=1
        if zero_x > zerocross: 
            sinalfinal3[:,qsinais6] = sinalfinal2[:,x]
            qsinais6=qsinais6+1

    return sinalfinal3,janela6,picos6


################ Filtro Gradiente #################
  def filtro_gradiente(self,sinalfinal3,picos6,janela6):

    (n,qsinais7) = sinalfinal3.shape
    contasinais4 = 0
    janela7 = []
    picos7 = []


    for x in range(0,qsinais7):
        
        if max(abs(np.gradient(sinalfinal3[:,x]))) < gradiente:
                
                janela7.append(janela6[x])
                picos7.append(picos6[x])
                contasinais4=contasinais4+1
                    

    # Cria matriz sinalfinal4
    sinalfinal4 = np.zeros(((amostras+1),(contasinais4)))
                    
    # Coloca sinais filtrados por Diferença de Constante na matriz sinalfinal4
    qsinais8 = 0

    for x in range(0,qsinais7):
        
            if max(abs(np.gradient(sinalfinal3[:,x]))) < gradiente:
                
                    sinalfinal4[:,qsinais8] = sinalfinal3[:,x]
                    qsinais8=qsinais8+1 
                    
    
    sinalfinal = sinalfinal4
    qsinais2 = qsinais8

    return sinalfinal4,janela7,picos7

######################################################################

#######################################
####     Padronização do Sinal     ####
#######################################

class padroniza_sinal:
  def __init__(self):
    pass

  def padroniza(self,resultado) :
    media = np.mean(resultado)
    sinalpadrao = (resultado-media)/np.std(resultado) 
    return sinalpadrao

#########################################################################

#######################################
####         Plotar Gráficos       ####
#######################################


class ClassePlote:

  def __init__(self):
    pass


#########################################################
  # Função realiza o plote de vetor x e y
  # Parametros:
  # - VetorX
  # - VetorY
  # - Xlabel
  # - Ylabel
  # - legend 
#########################################################
  def ploteLinha(self, VetorX, VetorY, Xlabel, Ylabel, Legend):
    try:
      plt.plot(VetorX, VetorY) #, linestyle='--', color="k")  
      #plt.scatter(VetorX, VetorY, color="r", marker="o", label=Legend)     
      plt.xlabel(Xlabel)
      plt.ylabel(Ylabel)
      plt.title(Legend)
      plt.grid(False)
      plt.show()
    except:
      print("Erro no metodo ploteLinha: na Linha {}".format(sys.exc_info()[-1].tb_lineno))
    
    
#########################################################
#Função realiza o plote histograma 1D
# Parametros: 
#########################################################
  def plotHistograma1D(self, Dado, Nbins, BoolDensity, Xlabel, Ylabel, Legend):
    try:
      bins =  OperadorPlot.hist(Dado, Nbins, density=BoolDensity, label=Legend)
      OperadorPlot.xlabel(Xlabel)
      OperadorPlot.ylabel(Ylabel)
      OperadorPlot.legend()       
      OperadorPlot.grid(True)
      OperadorPlot.show()
    except:
        print("Erro no metodo plotHistograma1D: na Linha {}".format(sys.exc_info()[-1].tb_lineno))
    
    
#########################################################
#Função realiza o plote de vetor x e y
# Parametros: 
#########################################################
  def getRandomVector(self, lenght, mu, sigma): 
    try:
        return (mu + sigma * np.random.randn(lenght))
    except:
        print("Erro no metodo getRandomVector: na Linha {}".format(sys.exc_info()[-1].tb_lineno))


#########################################################
#Função realiza o plote de sinais filtrados
# Parametros: 
#########################################################
  def plotar_sinais_filtrados(self,sinalfinal):
    qsinais2 = len(sinalfinal[0])
    linha = 0
    q = 0
    contador = 1
    linhas = range(0,(round(sinalfinal.shape[1]//2)))

    k= round((qsinais2/4)+1)
    if k == 1:
        k = 2
    k = round((qsinais2/4)+1)    


    #plt.rcParams['figure.figsize']=(10,10) 
    fig, ax = plt.subplots((k), 4,figsize=(16.4,8.6))
    fig.tight_layout()

    for x in range(0,(sinalfinal.shape[1])):
        
                                
            plt.rcParams['xtick.labelsize'] = 5
            plt.rcParams['ytick.labelsize'] = 5 
        
            
            
            plt.subplot(ax.shape[0],ax.shape[1],(x+1))
          
            plt.plot(range(1,(amostras+2)),sinalfinal[:,(x)])
            a=str(x+1)
            b=str(janela7[x])
            c=str(int(picos7[x]))
            d=' Coord '
            y=' Sinal '
            z= ' Máximo '
            titulo = d + b + y + a + z + c 
            plt.title(titulo,fontsize=12)
            plt.show





#########################################################################

#######################################
####         Identifica Threshold   ###
#######################################

class Ident_thres:
  def __init__(self):
    pass

  def threshold_sinal(self,opcao,amostras,sinalvet,limite):
    #%%%% a é o número de pontos do sinal
    #%%%% Delay são os número de pontos que irão retroagir
    #%%%% Amostras é o tamaho da janela dos sinais identificados
    #%%%% Limite é valor do threshlod para filtrar ruídos iniciais

    if opcao == 1:
        a1=delay + 30000 # + 3000 - RODO 
    else:
        a1=delay + 10000
        
    t=1  
    cont1=1
    janela=[]


    while (a1<(a-amostras)):
        if (abs(sinalvet[a1]) >= limite):

            #%%%% define a próxima janela a ser analisada
            janela.append(a1)
            a1=a1+amostras

        if (abs(sinalvet[a1]) < limite):

            #%%% desconsidera pontos abaixo do threshold e analisa o próximo ponto   
            a1=a1+1 

    return janela          


#########################################################################

#######################################
####         Destaca Sinais         ###
#######################################


class sinais:
  def __init__ (self):
    pass

  def  destaca_sinais(self,janela,amostras):

    k=len(janela) # quantidade de janelas                         
    sinalf=np.zeros(((amostras+1),k))
    picos=np.zeros(k)
    conta=0                    
    janela2=[]


    while (conta<=(k-1)):
    
        for i in janela:
            if i > delay: 
            
                
                #%%%% destaca o sinal do sinal principal e inclui no vetor sinalf         
                #sinalf[:,conta]=sinalvet[(i-delay):(i+amostras-delay+1)]
                pico=max(abs(sinalvet [(i-delay):(i+amostras-delay+1)]))

                for x in range ((i-delay),(i+amostras-delay+1)):
                  if abs(sinalvet[x]) == pico:
                    picos[conta]=x
                
                inicio = int(picos[conta]-(amostras/2))  
                fim = int(picos[conta]+(amostras/2)+1) 

                dif=fim-inicio 

                while dif != (amostras+1):
                  if dif < (amostras+1):
                    fim=fim+1
                    dif=fim-inicio 
                  if dif > (amostras+1):
                    fim=fim-1
                    dif=fim-inicio   


                sinalf[:,conta]= sinalvet[inicio:fim]  
                    
                #%%%% aumenta o contador para armazenar o próximo sinal no vetor
                conta = conta + 1
                janela2.append(i)
                #%%%% define a próxima janela a ser analisada
                #a1=a1+amostras
     
    return sinalf,janela2,picos



print("Classe carregada!")
print(datetime.datetime.now().strftime("%d-%m-%Y %H:%M:%S"))

Classe carregada!
20-08-2020 18:18:58


# **Algoritmo Principal**

In [50]:
plt.plot(sinalfinal2[:,3])

In [49]:
Path = "/content/drive/My Drive/Projeto Eddy Current/Sinais_Coletados/Banco de Dados/Medidas_SVTS/JI"
#Path = "/content/drive/My Drive/Projeto Eddy Current/Sinais_Coletados/Banco de Dados/6-Coleta de dados com o RD58 Ramal Paraibuna (vma20kmh) 04-09-2019"
File = "jti2_0.txt"
#File = "Dados_SVTS_10.txt"


#########################
# Definição de Variáveis#
#########################
wavelet = 0 # Se = 1 realiza a transformada wavelet
opcao=1  #1=sinal rodo  2=sinal manual
mediasup = 4.15 #0.55
mediainf = -6.15 #-0.35
zerocross = 2
tipo_de_grafico = 'flag'
escalas = 512
familia=['haar', 'db', 'sym', 'coif', 'bior', 'rbio', 'dmey', 'gaus', 'mexh', 'morl', 'cgau8', 'shan', 'fbsp', 'cmor']
familia2=['cgau1','cgau2','cgau3','cgau4','cgau5','cgau6','cgau7','cgau8','cmor','fbsp','gaus1','gaus2','gaus3','gaus4','gaus5','gaus6','gaus7','gaus8','mexh','morl','shan']
nome_da_wavelet = familia2[8]
#gradiente =  29 # 1.45 #3 sd  #9 manual #1.45 rodo
path ='C:/Projeto Eddy Current/Supervisionado/Sinais/teste/imagem'

######################
# Opções de aquisição#
######################

if opcao == 1: # RODO 

    limite = 1 #4 #threshold superior
    limite2 = -200 #threshold inferior
    delay=800 #% RODO
    amostras=1499 # RODO
    const=20 # RODO
    gradiente = 1
    

if opcao == 2:# MANUAL 
   
    limite = 0.2 #1.8 #4 #0.7 (Junta) #threshold superior #0.7
    limite2 = -200 #threshold inferior
    delay=3500 #500 #250(Junta)Rodo # manual 40
    amostras=9999#7999 #2499 #799(Junta) #Rodo # manual 399
    const=20 #Rodo
    gradiente = 1

#######################################################################

### Cria Operador Arquivo
OperadorArquivos = carrega_arquivo()
### Recebe o arquivo lido
sinal=OperadorArquivos.le_arquivo(Path,File)

x1=list(range(1,(len(sinal)+1)))

### Cria Operador Filtro
OperadorFiltro=filtro()
### Recebe o arquivo sem spikes
resultado=OperadorFiltro.tira_spikes(sinal)

### Cria Operador Padroniza Sinal
OperadorPadroniza = padroniza_sinal()
### Recebe o arquivo do sinal padronizado
sinalpadrao=OperadorPadroniza.padroniza(resultado) 

x2=list(range(1,(len(sinalpadrao)+1)))

### Cria Operador Plota Sinal
OperadorPlota = ClassePlote()
### Plota gráfico
OperadorPlota.ploteLinha(x1,sinal,'Amostras','Valor','Sinal Original ')
OperadorPlota.ploteLinha(x2,resultado,'Amostras','Valor','Sinal Original sem spikes')
OperadorPlota.ploteLinha(x2,sinalpadrao,'Amostras','Valor','Sinal Padronizado')

a=len(sinalpadrao)
sinalvet=sinalpadrao


### Cria Operador Threshold
OperadorThreshold = Ident_thres()
janela=OperadorThreshold.threshold_sinal(opcao,amostras,sinalpadrao,limite)

### Cria Operador Sinal
OperadorSinal=sinais()
### Destaca Sinais
(sinalf,janela2,picos)=OperadorSinal.destaca_sinais(janela,amostras)

### Filtra Sinal
### Filtro Média
(sinalfinal,janela4,picos4)=OperadorFiltro.filtro_media(sinalf,picos,janela2)
### Filtro FFT
(sinalfinal2,janela5,picos5)=OperadorFiltro.filtro_fft(sinalfinal,picos4,janela4)
### Filtro ZeroCross
(sinalfinal3,picos6,janela6)=OperadorFiltro.filtro_zerocross(sinalfinal2,picos5,janela5)
### Filtro Gradiente
(sinalfinal4,picos7,janela7)=OperadorFiltro.filtro_gradiente(sinalfinal3,picos6,janela6)
### Plota Sinais Filtrados
OperadorPlota.plotar_sinais_filtrados(sinalfinal4)


print(janela)
print(len(sinalf))
print(janela2)
print(picos)
print(len(sinalfinal))
print(janela4)
print(picos4)
print(len(sinalfinal2))
print(janela5)
print(picos5)
print(len(sinalfinal3))
print(janela6)
print(picos6)
print(len(sinalfinal4))
print(janela7)
print(picos7)
plt.plot(sinalfinal4[:,4])







<IPython.core.display.Javascript object>

[30807, 32306, 34233, 35749, 37248, 38747, 55012, 56578, 58342, 68612, 70176, 71675, 86301, 87871, 89442, 99454, 101413, 102947, 122182, 123772, 125536, 137883, 139487, 140988, 159404, 160903, 162421, 173768, 175267, 176847, 193757, 195256, 196755, 198254, 199924, 201423, 202922, 204421, 205952, 207451, 208950, 210492, 211991, 213490, 230178, 231686, 233185, 234809, 236308, 243686, 245189, 246688, 264569, 266076, 267659, 269158, 271614, 273903, 275402, 276901, 278400, 279899, 281671, 283285, 301167, 302669, 304314, 305835, 316302, 317805, 319304, 338610, 340157, 342093, 343592, 345197]
1500
[30807, 32306, 34233, 35749, 37248, 38747, 55012, 56578, 58342, 68612, 70176, 71675, 86301, 87871, 89442, 99454, 101413, 102947, 122182, 123772, 125536, 137883, 139487, 140988, 159404, 160903, 162421, 173768, 175267, 176847, 193757, 195256, 196755, 198254, 199924, 201423, 202922, 204421, 205952, 207451, 208950, 210492, 211991, 213490, 230178, 231686, 233185, 234809, 236308, 243686, 245189, 246688, 2